# مقایسه آمار مرگ و میر ایران با باقی کشورها
> مقایسه مرگ و میر ایران با کشورهای ایتالیا ، کره جنوبی ، ژاپن و آمریکا.

- comments: false
- toc: true
- author:  پراتاب واردهان
- categories: [افرایش, مقایسه, مرگ و میر]
- image: images/covid-compare-country-death-trajectories.png
- permalink: /compare-country-death-trajectories/

In [1]:
#hide
!pip install jdatetime
!pip install persiantools
import pandas as pd
import altair as alt
import jdatetime
from persiantools import digits
from IPython.display import HTML

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
#hide
url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df = pd.read_csv(url)
# rename countries
df['Country/Region'] = df['Country/Region'].replace({'Korea, South': 'South Korea'})
# Add countries
countries = ["China", "Italy", "Spain", "France", "United Kingdom", "Germany", 
             "US", "South Korea", "Japan", 
             "Iran", "India", "Canada", "Australia"]
df = df[df['Country/Region'].isin(countries)]   # Just use list above
df['Country/Region'] = df['Country/Region'].replace({'Italy': 'ایتالیا'})
df['Country/Region'] = df['Country/Region'].replace({'China': 'چین'})
df['Country/Region'] = df['Country/Region'].replace({'Spain': 'اسپانیا'})
df['Country/Region'] = df['Country/Region'].replace({'France': 'فرانسه'})
df['Country/Region'] = df['Country/Region'].replace({'Germany': 'آلمان'})
df['Country/Region'] = df['Country/Region'].replace({'United Kingdom': 'انگلیس'})
df['Country/Region'] = df['Country/Region'].replace({'US': 'آمریکا'})
df['Country/Region'] = df['Country/Region'].replace({'South Korea': 'کره جنوبی'})
df['Country/Region'] = df['Country/Region'].replace({'Japan': 'ژاپن'})
df['Country/Region'] = df['Country/Region'].replace({'Iran': 'ایران'})
df['Country/Region'] = df['Country/Region'].replace({'India': 'هند'})
df['Country/Region'] = df['Country/Region'].replace({'Canada': 'کانادا'})
df['Country/Region'] = df['Country/Region'].replace({'Australia': 'استرالیا'})
dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

In [3]:
#hide
url_italy = ('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv')
df_italy = pd.read_csv(url_italy)

In [4]:
#hide
dff = (df.groupby('Country/Region')[dt_cols].sum()
       .stack().reset_index(name='Confirmed Cases')
       .rename(columns={'level_1': 'Date', 'Country/Region': 'Country'}))
dff['Date'] = pd.to_datetime(dff['Date'], format='%m/%d/%y')

In [5]:
#hide
MIN_CASES = 10
LAST_DATE = dt_cols[-1]
# sometimes last column may be empty, then go backwards
for c in dt_cols[::-1]:
    if not df[c].fillna(0).eq(0).all():
        LAST_DATE = c
        break
countries = dff[dff['Date'].eq(LAST_DATE) & dff['Confirmed Cases'].ge(MIN_CASES) & 
        dff['Country'].ne('China')
       ].sort_values(by='Confirmed Cases', ascending=False)
countries = countries['Country'].values

In [6]:
#hide
# more regions of Italy can be added alongside Lombardia in the next line of code (ex: 'Emilia Romagna', 'Veneto', etc.)
df_reg = df_italy[df_italy['denominazione_regione'].isin(['Lombardia'])]
df_reg = df_reg.sort_values(by=['denominazione_regione', 'deceduti'])
mask = (df_reg['deceduti'] >= MIN_CASES)
df_reg_gt10 = df_reg.loc[mask]
df_simple = df_reg_gt10[['denominazione_regione','data', 'deceduti']]
df_simple.rename(columns={'denominazione_regione':'Country'}, inplace=True)
df_simple.rename(columns={'data':'Date'}, inplace=True)
df_simple.rename(columns={'deceduti':'Confirmed Cases'}, inplace=True)
df_simple['Date'] = pd.to_datetime(df_simple["Date"]).dt.strftime('%Y-%m-%d')
df_simple['Date'] = pd.to_datetime(df_simple["Date"])

In [7]:
#hide
df_simple[f'Days since {MIN_CASES}th death'] = ''
current_region = df_simple.iloc[0]['Country']
days = 0
for index, row in df_simple.iterrows():
  if row['Country'] == current_region:
    df_simple[f'Days since {MIN_CASES}th death'][index] = days
    days += 1
  else:
    days = 0
    current_region = df_simple['Country'][index]
    df_simple[f'Days since {MIN_CASES}th death'][index] = days
    days += 1

df_simple["Country"]= df_simple["Country"].replace("Lombardia", "لامباردی")

In [8]:
#hide
SINCE_CASES_NUM = 10
COL_X = f'Days since {SINCE_CASES_NUM}th death'
dff2 = dff[dff['Country'].isin(countries)].copy()
days_since = (dff2.assign(F=dff2['Confirmed Cases'].ge(SINCE_CASES_NUM))
              .set_index('Date')
              .groupby('Country')['F'].transform('idxmax'))
dff2[COL_X] = (dff2['Date'] - days_since.values).dt.days.values
dff2 = dff2[dff2[COL_X].ge(0)]

# append Italy data to the rest of the data
dff2 = dff2.append(df_simple)

In [9]:
#hide
def get_country_colors(x):
    mapping = {
        'ایران': 'black',
        'ایتالیا': '#A569BD',
        'چین': '#A1BA59',
        'اسپانیا': '#D5F5E3',
        'آمریکا': '#FF5733', 
        'کره جنوبی': '#A9DFBF',
        'ژاپن': '#A9CCE3',
        'فرانسه': '#EB984E',
        'آلمان': '#E59866',
        'انگلیس': '#D7BDE2',
        'هند': '#C1B7AD',
        'کانادا': '#F9E79F',
        'استرالیا': '#ABEBC6'}
    return mapping.get(x, '#C1B7AD')

In [10]:
#hide_input
baseline_countries = ['ایتالیا', 'کره جنوبی', 'ژاپن','آمریکا','ایران']
max_date = dff2['Date'].max()
color_domain = list(dff2['Country'].unique())
color_range = list(map(get_country_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['Country'], bind='legend', 
                                    init=[{'Country': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=550).encode(
        x=alt.X(f'{COL_X}:Q',axis=alt.Axis(titleFont='Tahoma',title='تعداد روز سپری شده از ۱۰ مورد مرگ اول')),
        y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(titleFont='Tahoma',title='مرگ و میر تجمعی')),
        color=alt.Color(
            'Country:N',
            scale=alt.Scale(domain=color_domain, range=color_range),
            legend=alt.Legend(columns=5, title='کشور',labelFont='Tahoma',labelFontSize=13,titleFontSize=14, titleFont='Tahoma',symbolLimit=100, orient='bottom')),
        tooltip=list(dff2),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
 
    max_day = dff2[COL_X].max()
    ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'Confirmed Cases'])
    base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='Confirmed Cases:Q')
   
    return (
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) +
        base_ref.transform_filter(
            alt.datum[COL_X] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=14, color='#FF5733' ,text='۳۳٪ رشد روزانه') +
        base.mark_line(point=True).add_selection(selection) + 
        base.transform_filter(
            alt.datum['Date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='Country:N')
    ).properties(
        title=f"مقایسه رشد مرگ و میر در ایران با {', '.join(baseline_countries)}"
    ).configure_title(fontSize=15, font='Tahoma', color='#24292e').configure_axis(
    grid=False
    ).configure_view(
    strokeOpacity=0
    )
    

##  یادگیری از ایتالیا ، کره جنوبی ،  آمریکا و ژاپن

ایتالیا ، کره جنوبی و ژاپن سه کشوری هستند که میزان رشد مرگ و میر در طی زمان در آنها با هم متفاوت است.

**ایتالیا** ، **کره جنوبی** ، **ژاپن** و **آمریکا**  رشد مرگ و میر در طی زمان در آنها با هم متفاوت بوده است.
کره جنوبی و ژاپن بعذ از ۱۰ امین مورد مرگ ، سرعت رشد کمتری نشان داده اند. **ایتالیا** تا سه هفته بعد از زمان ۱۰ امین مورد مرگ خود ، نزدیک به ۳۳٪ رشد داشته اند. 

ایران چگونه است؟

برای مشاهده چند مورد باهم دکمه شیفت را فشار دهید.


In [11]:
#hide_input
gredate = pd.to_datetime(LAST_DATE)
jalili_date = jdatetime.date.fromgregorian(year=gredate.year,month=gredate.month,day=gredate.day,locale='fa_IR').strftime("%d / %m / %Y")
jalili_date = digits.en_to_fa(jalili_date)
HTML(f'<small class="float-right jdate"> آخرین بروزرسانی در :   {jalili_date} </small>')

In [12]:
#hide_input
chart = make_since_chart()
chart

alt.LayerChart(...)

کشور مورد نظر خود را انتخاب کنید:

In [13]:
#hide_input
base = alt.Chart(dff2, width=600).encode(
    x=alt.X(f'{COL_X}:Q',axis=alt.Axis(titleFont='Tahoma',title='تعداد روز سپری شده از ۱۰ مورد مرگ اول')),
    y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(titleFont='Tahoma', title='مرگ و میر تجمعی')),
    color=alt.Color('Country:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),
    tooltip=['Country', 'Date', 'Confirmed Cases', COL_X]
)

country_selection = alt.selection_single(
    name='Select', fields=['Country'], 
    bind=alt.binding_select(options=list(sorted(set(countries) - set(baseline_countries)))),
    init={'Country': 'ایران'})

date_filter = alt.datum['Date'] >= int(max_date.timestamp() * 1000)
base2 = base.transform_filter(alt.FieldOneOfPredicate('Country', baseline_countries))
base3 = base.transform_filter(country_selection)
base4 = base3.transform_filter(date_filter)

max_day = dff2[COL_X].max()
ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'Confirmed Cases'])
base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='Confirmed Cases:Q')
base_ref_f = base_ref.transform_filter(alt.datum[COL_X] >= max_day)

chart5 = (
 base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) + 
 base_ref_f.mark_text(dy=-6, align='right', fontSize=14, color='#FF5733', text='۳۳٪ رشد روزانه') + 
 base2.mark_line(point=True, tooltip=True) +
 base3.mark_line(point={'size':50}, tooltip=True) +
 base2.transform_filter(date_filter).mark_text(dy=-8, align='right').encode(text='Country:N') +
 base4.mark_text(dx=8, align='left', fontWeight='bold').encode(text='Country:N') +
 base4.mark_text(dx=8, dy=12, align='left', fontWeight='bold').encode(text='Confirmed Cases:Q')
).add_selection(country_selection).properties(
    title=f"مقایسه مرگ و میر با {', '.join(baseline_countries)}"
).configure_title(fontSize=15, font='Tahoma', color='#24292e').configure_axis(
    grid=False
    ).configure_view(
    strokeOpacity=0
    )
chart5

alt.LayerChart(...)

## در لامباردی چه گذشت؟

لومباردی ، منطقه ایی در ایتالیا با تنها ۱۰ میلیون نفر جمعیت ، دارای مرگ و میر بیش از ۴۰۰۰ تن به دلیل کووید ۱۹ است. لومباردی در بالاترین میزان مرگ و میر روزانه خود ، در یک روز ۵۴۶ کشته برجای گذاشت. این نشان می دهد که ویروس حتی در مقیاس جمعیتی کوچک ، احتمالاً به دلیل خدمات مراقبت های بهداشتی اندک میتواند میزان مرگ و میر بالایی داشته باشد.

ایران در مقایسه با لمباردی کجا ایستاده است؟


In [14]:
#hide_input
chart5 = make_since_chart(['انگلیس','ایران',  'آمریکا', 'اسپانیا', 'آلمان', 'کره جنوبی'], ['لامباردی'])
chart5

alt.LayerChart(...)


منحنی مقایسه با لامباردی توسط  [ولن استان](https://twitter.com/vladpke)[^۲]

تحلیل اصلی از [پراتاپ واردهان](https://twitter.com/PratapVardhan)[^۱]

[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).

[^۲]: منبع: ["Dati COVID-19 Italia"](http://www.salute.gov.it/portale/home.html) [GitHub repository](https://github.com/pcm-dpc/COVID-19).